<a href="https://colab.research.google.com/github/alorenz465446/NYC_Parking_Tickets/blob/main/Condensed_Random_Forest_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
import pandas as pd
import io

# Add code to import from computer to google colab. 
from google.colab import files
uploaded = files.upload()



Saving dropped_na_nyc_color_fix.csv to dropped_na_nyc_color_fix (1).csv


In [ ]:
nyc_df = pd.read_csv(io.BytesIO(uploaded['dropped_na_nyc_color_fix.csv']))
nyc_df 

,Plate_Type,Violation_Code,Vehicle_Body_Type,Vehicle_Color,Vehicle_Year,Violation_County,Violation_Time
0,PAS,21.0,SUBN,SILVER,2008,K,0950A
1,PAS,38.0,SUBN,WHITE,2014,NY,0817A
2,PAS,21.0,4DSD,BLACK,0,Q,0148A
3,PAS,21.0,4DSD,RED,0,K,0820A
4,PAS,36.0,4DSD,GREY,2011,QN,1047A
...,...,...,...,...,...,...,...
338518,PAS,21.0,SUBN,GREEN,2004,Q,1201P
338519,PAS,14.0,SUBN,BLACK,0,Q,1046A
338520,PAS,36.0,4DSD,RED,2003,QN,0200P
338521,PAS,38.0,4DSD,BLUE,2015,K,0202P


In [ ]:
list(nyc_df.columns)

['Plate_Type',
 'Violation_Code',
 'Vehicle_Body_Type',
 'Vehicle_Color',
 'Vehicle_Year',
 'Violation_County',
 'Violation_Time']

In [ ]:
nyc_df.dtypes

Plate_Type            object
Violation_Code       float64
Vehicle_Body_Type     object
Vehicle_Color         object
Vehicle_Year           int64
Violation_County      object
Violation_Time        object
dtype: object

In [ ]:
# drop Violation_Time column
nyc_df = nyc_df.drop("Violation_Time", axis=1)

In [ ]:
nyc_df['Vehicle_Color'].value_counts()

GREY      102581
BLACK      90550
WHITE      60510
BLUE       36250
RED        23533
SILVER     12674
GREEN      11477
YELLOW       508
ORANGE       230
PURPLE       210
Name: Vehicle_Color, dtype: int64

In [ ]:
# Group colors together:
def group_colors(color):
  if color == 'GREY':
    return "DULL"
  elif color == 'WHITE':
    return "DULL"
  elif color == 'BLACK':
    return "DULL"
  elif color == 'SILVER':
    return "DULL"
  elif color == 'BLUE':
    return "BRIGHT"
  elif color == 'RED':
    return "BRIGHT"
  elif color == 'GREEN':
    return "BRIGHT"
  elif color == 'YELLOW':
    return "BRIGHT"
  elif color == 'ORANGE':
    return "BRIGHT"
  elif color == 'PURPLE':
    return "BRIGHT"

nyc_df['Vehicle_Color']= nyc_df['Vehicle_Color'].apply(group_colors)


# Look at 'Vehicle_Color' value counts for binning
Vehicle_Color_count=nyc_df['Vehicle_Color'].value_counts()
Vehicle_Color_count

DULL      266315
BRIGHT     72208
Name: Vehicle_Color, dtype: int64

In [ ]:
# checkout Violation_Code column 
nyc_df['Violation_Code'].value_counts()


21.0    101941
36.0     87992
38.0     70018
20.0     42380
14.0     36192
Name: Violation_Code, dtype: int64



*   21	Street Cleaning: No parking where parking is not allowed by sign, street marking or traffic control device.
*   36	Exceeding the posted speed limit in or near a designated school zone.
*   38	Failing to show a receipt or tag in the windshield.
*   20	General No Parking: No parking where parking is not allowed by sign, street marking or traffic control device.
*   14	General No Standing: Standing or parking where standing is not allowed by sign, street marking or; traffic control device.


# keep 38, bin 20 and 14 
# prediction 
no parking gets ticketed 




In [ ]:
# def lable_tickets(ticket):
#   if ticket == 14.0 or ticket == 20:
#     return 'No Parking/Standing'
#   elif ticket == 38.0:
#     return 'No receipt in windshield'
#   else: return 'other'

# nyc_df['Violation_Code']= nyc_df['Violation_Code'].apply(lable_tickets)
# nyc_df

# Encode Categorical Data 
Use one One_Hot_Encoder 

In [ ]:
# Generate our categorical variable list
df_categorical = nyc_df.dtypes[nyc_df.dtypes == "object"].index.tolist()
df_categorical



['Plate_Type', 'Vehicle_Body_Type', 'Vehicle_Color', 'Violation_County']

In [ ]:
nyc_df['Plate_Type'].value_counts()

PAS    333754
COM      4769
Name: Plate_Type, dtype: int64

In [ ]:
# drop commercial vehicles
nyc_df= nyc_df[nyc_df.Plate_Type != 'COM']  
nyc_df

,Plate_Type,Violation_Code,Vehicle_Body_Type,Vehicle_Color,Vehicle_Year,Violation_County
0,PAS,21.0,SUBN,DULL,2008,K
1,PAS,38.0,SUBN,DULL,2014,NY
2,PAS,21.0,4DSD,DULL,0,Q
3,PAS,21.0,4DSD,BRIGHT,0,K
4,PAS,36.0,4DSD,DULL,2011,QN
...,...,...,...,...,...,...
338518,PAS,21.0,SUBN,BRIGHT,2004,Q
338519,PAS,14.0,SUBN,DULL,0,Q
338520,PAS,36.0,4DSD,BRIGHT,2003,QN
338521,PAS,38.0,4DSD,BRIGHT,2015,K


In [ ]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(nyc_df[df_categorical]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(df_categorical)
encode_df.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Vehicle_Color_BRIGHT,Vehicle_Color_DULL,Violation_County_BK,Violation_County_BX,Violation_County_K,Violation_County_MN,Violation_County_NY,Violation_County_Q,Violation_County_QN,Violation_County_R,Violation_County_ST
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# Merge one-hot encoded features and drop the originals
nyc_df = nyc_df.merge(encode_df,left_index=True, right_index=True)
nyc_df = nyc_df.drop(df_categorical,1)
nyc_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Violation_Code,Vehicle_Year,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Vehicle_Color_BRIGHT,Vehicle_Color_DULL,Violation_County_BK,Violation_County_BX,Violation_County_K,Violation_County_MN,Violation_County_NY,Violation_County_Q,Violation_County_QN,Violation_County_R,Violation_County_ST
0,21.0,2008,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38.0,2014,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,21.0,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,21.0,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,36.0,2011,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Start with a Random Forest model to check for correlations. 

Note:
(used homework 17.8.2-3, named random_forest_model in JN)
- BM

***need to create a sampling of the data to use RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


### Preprocess the Data


In [ ]:
list(nyc_df.columns )


['Violation_Code',
 'Vehicle_Year',
 'Plate_Type_PAS',
 'Vehicle_Body_Type_2DSD',
 'Vehicle_Body_Type_4DSD',
 'Vehicle_Body_Type_PICK',
 'Vehicle_Body_Type_SUBN',
 'Vehicle_Color_BRIGHT',
 'Vehicle_Color_DULL',
 'Violation_County_BK',
 'Violation_County_BX',
 'Violation_County_K',
 'Violation_County_MN',
 'Violation_County_NY',
 'Violation_County_Q',
 'Violation_County_QN',
 'Violation_County_R',
 'Violation_County_ST']

In [ ]:
# Define the features set.
X = nyc_df.copy()
X = X.drop(['Violation_Code'], axis=1)
X.head()


,Vehicle_Year,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Vehicle_Color_BRIGHT,Vehicle_Color_DULL,Violation_County_BK,Violation_County_BX,Violation_County_K,Violation_County_MN,Violation_County_NY,Violation_County_Q,Violation_County_QN,Violation_County_R,Violation_County_ST
0,2008,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# Define the target set.
y = nyc_df["Violation_Code"].ravel()
y[:5]


array([21., 38., 21., 21., 36.])

In [ ]:
# Split data into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Fit the Random Forest Model


In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128) 


> The `n_estimators` will allow us to set the number of trees that will be created by the algorithm. Generally, the higher number makes the predictions stronger and more stable, but can slow down the output because of the higher training time allocated. The best practice is to use between 64 and 128 random forests, though higher numbers are quite common despite the higher training time. 

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


### Make Predictions Using the Testing Data


In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
list(predictions)

[21.0,
 36.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 36.0,
 21.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 36.0,
 21.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 21.0,
 36.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 36.0,
 21.0,
 36.0,
 36.0,
 36.0,
 38.0,
 21.0,
 21.0,
 36.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 21.0,
 21.0,
 36.0,
 21.0,
 21.0,
 36.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 21.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 21.0,
 36.0,
 36.0,
 21.0,
 21.0,
 36.0,
 21.0,
 36.0,
 36.0,
 21.0,
 36.0,
 36.0,
 36.0,
 36.0,
 36.0,

### Evaluate the Model


21.0    101941
36.0     87992
38.0     70018
20.0     42380
14.0     36192

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=['Actual 21.0', 36.0, 38.0, 20.0, 14.0], columns=["Predicted: Street Cleaning", "Speeding", "No Receipt", "No Parking", "No Standing"])

cm_df

,Predicted: Street Cleaning,Speeding,No Receipt,No Parking,No Standing
Actual /br 21.0,6,19,4480,4001,49
36.0,8,26,5267,5086,44
38.0,25,32,12656,11927,129
20.0,17,57,5557,15923,130
14.0,23,21,7242,9435,106


In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)


In [ ]:
# Displaying results
# print("Confusion Matrix")
# display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("-")
print("Classification Report")
print(classification_report(y_test, predictions))


Accuracy Score : 0.3490749519850242
-
Classification Report
              precision    recall  f1-score   support

        14.0       0.08      0.00      0.00      8555
        20.0       0.17      0.00      0.00     10431
        21.0       0.36      0.51      0.42     24769
        36.0       0.34      0.73      0.47     21684
        38.0       0.23      0.01      0.01     16827

    accuracy                           0.35     82266
   macro avg       0.24      0.25      0.18     82266
weighted avg       0.28      0.35      0.25     82266



### Rank the Importance of Features


In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


In [ ]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


# Neuro Network 
We will be building a neuro network. First we need to clean the data, see correlations, and then make the plan for the neural network. 